In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import json
import pickle

import src.bikewaysim_functions as bikewaysim_functions

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

In [ ]:
#study area
studyarea = gpd.read_file(config['studyarea_fp'])

arc_fp = Path("D:\RAW\Atlanta_Regional_Comission")

# how many tazs
print(gpd.read_file(arc_fp/"Model_Traffic_Analysis_Zones_2020.geojson",ignore_geometry=True).shape[0])

#import taz geo
tazs = gpd.read_file(arc_fp/"Model_Traffic_Analysis_Zones_2020.geojson",mask=studyarea)[['MTAZ10','geometry']]
print(tazs.shape[0])
# tazs['centroid_geo'] = tazs.geometry.centroid
# tazs.set_geometry('centroid_geo',inplace=True)

In [ ]:
#export the taz areas
tazs.to_crs(config['projected_crs_epsg']).to_file(config['calibration_fp']/'map_layers.gpkg',layer='taz_polygons')
#export the taz centroids
taz_centroids = tazs.copy()
taz_centroids.geometry = taz_centroids.geometry.centroid
taz_centroids.to_crs(config['projected_crs_epsg']).to_file(config['calibration_fp']/'map_layers.gpkg',layer='taz_centroids')

In [ ]:
# import network nodes
nodes = gpd.read_file(config['network_fp']/"final_network.gpkg",layer="nodes")

with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links,turns = pickle.load(fh)
del turns

nodes0 = links['A'].append(links['B'])
nodes = nodes[nodes['N'].isin(nodes0)]

In [ ]:
# snap tazs to network
tazs = bikewaysim_functions.snap_ods_to_network(tazs,nodes)

# only keep if dist is less than or equal to 500 feet
tazs = tazs[tazs['dist']<=500]

In [ ]:
#import arc trip data
tripData0 = pd.read_csv(arc_fp/"rtp_amd6_2030_tripData/tripData.csv",usecols=['orig_taz','dest_taz','distance'])
tripData1 = pd.read_csv(arc_fp/"rtp_amd6_2030_tripData/tripData_1.csv",usecols=['orig_taz','dest_taz','distance'])
tripData2 = pd.read_csv(arc_fp/"rtp_amd6_2030_tripData/tripData_2.csv",usecols=['orig_taz','dest_taz','distance'])
tripData = pd.concat([tripData0,tripData1,tripData2])
del tripData0, tripData1, tripData2
tripData = tripData[tripData['distance'] <= 3]
tripData = tripData.groupby(['orig_taz','dest_taz']).size().reset_index()
tripData.columns = ['orig_taz','dest_taz','trips']
tripData = tripData[tripData['orig_taz'].isin(tazs['MTAZ10']) & tripData['dest_taz'].isin(tazs['MTAZ10'])]
tripData = tripData[tripData['orig_taz'] != tripData['dest_taz']]

In [ ]:
print(tripData['trips'].sum(),'trips across',tripData.shape[0],'unique OD pairs')

In [ ]:
snapped_ods = dict(zip(tazs['MTAZ10'],tazs['N']))

tripData['orig_N'] = tripData['orig_taz'].map(snapped_ods)
tripData['dest_N'] = tripData['dest_taz'].map(snapped_ods)

In [ ]:
tripData

In [ ]:
#export
tripData.to_csv(config['bikewaysim_fp']/'od_matrix.csv',index=False)
#tazs.to_file(bikewaysim_fp/'snapped_ods.gpkg')

#export the centroid version too

In [ ]:
tripData0 = pd.read_csv(arc_fp/"rtp_amd6_2030_tripData/tripData.csv",nrows=10)

In [ ]:
tripData0.columns